In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from glob import glob
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
import math
from sklearn.preprocessing import LabelEncoder
from google.colab import drive
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
url = os.chdir(r"/content/drive/MyDrive/hackatonZP/")

In [5]:
df = pd.read_parquet('/content/drive/MyDrive/hackatonZP/test_dataset_hackaton2023_test.gzip')

In [6]:
a = LabelEncoder()
df['dish_name'] = a.fit_transform(df['dish_name'])
b = LabelEncoder()
df['format_name'] = b.fit_transform(df['format_name'])

In [29]:
empty = np.array([0] * len(df['customer_id'].unique()[:100]))
df_test = pd.DataFrame({
    'customer_id': empty,
    'popular_dish': empty,
    'avg_revenue': empty,
    'avg_sqm': empty,
    'popular_format': empty,
    'm1_dish': empty,
    'm2_dish': empty,
    'm1_sqm': empty,
    'm2_sqm': empty,
    'm1_revenue': empty,
    'm2_revenue': empty,
    'm2_interval': empty,
    'mean_delta_time': empty,
    'visits': empty
})
df_test = df_test.reset_index()

for i, customer in tqdm(enumerate(df['customer_id'].unique()[:100])):
    customer = groups.get_group(customer).reset_index()
    df_test.at[i, 'customer_id'] = customer.at[0, 'customer_id']

    meals = customer.groupby('startdatetime')
    pt = 0
    for j, meal in enumerate(customer['startdatetime'].unique()):
        meal = meals.get_group(meal).reset_index()
        if j > 1:
            break
        df_test.at[i, f'm{j+1}_dish'] = meal['dish_name'].value_counts().idxmax()
        df_test.at[i, f'm{j+1}_revenue'] =  meal['revenue'].sum()
        df_test.at[i, f'm{j+1}_sqm'] = meal['ownareaall_sqm'].iloc[0]
        if j > 0:
            df_test.at[i, f'm{j+1}_interval'] = (meal['startdatetime'].iloc[0] - pt) / np.timedelta64(1, 'D')
        pt = meal['startdatetime'].iloc[0]

    df_test.at[i, 'visits'] = len(customer['startdatetime'].unique())
    df_test.at[i, 'mean_interval'] = (customer['startdatetime'].iloc[len(customer['startdatetime']) - 1] - customer['startdatetime'].iloc[0]) / np.timedelta64(1, 'D') / df_test.at[i, 'visits']

    df_test.at[i, 'avg_revenue'] = customer['revenue'].mean()
    df_test.at[i, 'avg_sqm'] = customer['ownareaall_sqm'].mean()

    df_test.at[i, 'popular_dish'] = customer['dish_name'].value_counts().idxmax()

    df_test.at[i, 'popular_format'] = customer['format_name'].value_counts().idxmax()
df_test

100it [00:01, 92.96it/s]


,index,customer_id,popular_dish,avg_revenue,avg_sqm,popular_format,m1_dish,m2_dish,m1_sqm,m2_sqm,m1_revenue,m2_revenue,m2_interval,mean_delta_time,visits,mean_interval
0,0,13220760,666,40.141587,175.790476,8,607,178,174.40,74.1,2.00,419.98,-15.761007,0,11,-0.991379
1,1,30315975,789,142.490000,235.000000,1,789,845,235.00,235.0,584.96,144.98,-26.936829,0,3,0.000000
2,2,21679985,666,48.471053,209.526316,7,666,666,165.00,165.0,274.98,349.99,16.060093,0,8,5.267212
3,3,29754274,899,112.202632,98.421053,7,899,833,150.00,80.0,509.96,690.95,-21.969896,0,4,-5.492474
4,4,7797823,666,71.571529,114.500000,7,94,666,114.50,114.5,468.97,520.96,11.252500,0,12,2.748228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,3296941,916,84.781053,403.105263,6,666,916,431.10,415.6,409.98,619.93,6.965475,0,5,-1.450231
96,96,22191359,666,44.346047,106.704651,7,666,666,115.46,86.5,642.95,501.98,-37.359583,0,4,0.000000
97,97,21436114,925,86.171765,114.400000,8,924,899,114.40,114.4,249.99,484.97,-6.199167,0,4,5.948573
98,98,8369200,934,153.075510,297.000000,2,542,30,297.00,297.0,1979.92,2.00,22.095613,0,9,-2.421672


In [30]:
from joblib import load
clf = load('filename.joblib')
clf

HistGradientBoostingClassifier()

In [33]:
y = clf.predict(df_test.drop(columns=['customer_id','index']))
y

array([0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0])

In [40]:
X_reg1 = df_test.drop(columns=['customer_id','index'])
X_reg1['buy_post'] = y
X_reg1

,popular_dish,avg_revenue,avg_sqm,popular_format,m1_dish,m2_dish,m1_sqm,m2_sqm,m1_revenue,m2_revenue,m2_interval,mean_delta_time,visits,mean_interval,buy_post
0,666,40.141587,175.790476,8,607,178,174.40,74.1,2.00,419.98,-15.761007,0,11,-0.991379,0
1,789,142.490000,235.000000,1,789,845,235.00,235.0,584.96,144.98,-26.936829,0,3,0.000000,0
2,666,48.471053,209.526316,7,666,666,165.00,165.0,274.98,349.99,16.060093,0,8,5.267212,1
3,899,112.202632,98.421053,7,899,833,150.00,80.0,509.96,690.95,-21.969896,0,4,-5.492474,0
4,666,71.571529,114.500000,7,94,666,114.50,114.5,468.97,520.96,11.252500,0,12,2.748228,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,916,84.781053,403.105263,6,666,916,431.10,415.6,409.98,619.93,6.965475,0,5,-1.450231,0
96,666,44.346047,106.704651,7,666,666,115.46,86.5,642.95,501.98,-37.359583,0,4,0.000000,0
97,925,86.171765,114.400000,8,924,899,114.40,114.4,249.99,484.97,-6.199167,0,4,5.948573,1
98,934,153.075510,297.000000,2,542,30,297.00,297.0,1979.92,2.00,22.095613,0,9,-2.421672,0


In [46]:
reg = load('regressor.joblib')
reg

RandomForestRegressor()

In [71]:
X_reg1['date_diff_post'] = reg.predict(df_test.drop(columns=['customer_id','index']))
X_reg1.loc[X_reg1['buy_post']==0, 'date_diff_post'] = np.NaN
X_reg1['id'] = df_test['customer_id']
X_reg1

,popular_dish,avg_revenue,avg_sqm,popular_format,m1_dish,m2_dish,m1_sqm,m2_sqm,m1_revenue,m2_revenue,m2_interval,mean_delta_time,visits,mean_interval,buy_post,date_diff_post,id
0,666,40.141587,175.790476,8,607,178,174.40,74.1,2.00,419.98,-15.761007,0,11,-0.991379,0,NaN,13220760
1,789,142.490000,235.000000,1,789,845,235.00,235.0,584.96,144.98,-26.936829,0,3,0.000000,0,NaN,30315975
2,666,48.471053,209.526316,7,666,666,165.00,165.0,274.98,349.99,16.060093,0,8,5.267212,1,17.38,21679985
3,899,112.202632,98.421053,7,899,833,150.00,80.0,509.96,690.95,-21.969896,0,4,-5.492474,0,NaN,29754274
4,666,71.571529,114.500000,7,94,666,114.50,114.5,468.97,520.96,11.252500,0,12,2.748228,1,21.80,7797823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,916,84.781053,403.105263,6,666,916,431.10,415.6,409.98,619.93,6.965475,0,5,-1.450231,0,NaN,3296941
96,666,44.346047,106.704651,7,666,666,115.46,86.5,642.95,501.98,-37.359583,0,4,0.000000,0,NaN,22191359
97,925,86.171765,114.400000,8,924,899,114.40,114.4,249.99,484.97,-6.199167,0,4,5.948573,1,34.65,21436114
98,934,153.075510,297.000000,2,542,30,297.00,297.0,1979.92,2.00,22.095613,0,9,-2.421672,0,NaN,8369200


In [72]:
otvet = X_reg1[['id','buy_post','date_diff_post']]
otvet

,id,buy_post,date_diff_post
0,13220760,0,NaN
1,30315975,0,NaN
2,21679985,1,17.38
3,29754274,0,NaN
4,7797823,1,21.80
...,...,...,...
95,3296941,0,NaN
96,22191359,0,NaN
97,21436114,1,34.65
98,8369200,0,NaN


In [73]:
otvet.to_csv('ans.csv')